In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from pytorch_tabnet.multitask import TabNetMultiTaskClassifier
from pytorch_tabnet.metrics import Metric

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np

import datetime
from datetime import datetime as dt

import requests
import json
import csv

import warnings
warnings.filterwarnings('ignore')

C:\Users\rian\miniconda3\envs\nonmun\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def engineering(n, d):
    # 성별 인코딩
    d['gender'] = d['gender'].apply(lambda x:0 if x == 'M' else 1)
    # 날짜 변환
    d['date'] = pd.to_datetime(d['date'])
    # 스트레스(target)
#     d['pmStress'] = d['pmStress'] - 1
    
    
    # 이 아래부터는 다 쓸 필요는 없고, 결과나 편의에 따라서 사용하시면 됩니다!
    
    # 감정변화비율 = 오후감정/오전감정
    d['emotionChangeRate'] = d['pmEmotion'] / d['amEmotion']
    # 긍정변화평균 = emotion1~7의 평균
    d['positiveMean'] = d.filter(regex='Positive').mean(axis=1)
    # 긴장변화평균 = emotion1~7의 평균
    d['tensionnMean'] = d.filter(regex='Tension').mean(axis=1)
    
    # 긍정비율 = Positive5~7 / 1~3
    d['positiveRate'] = d[['emotionPositive5', 'emotionPositive6', 'emotionPositive7']].sum(axis=1) / d[['emotionPositive1', 'emotionPositive2', 'emotionPositive3']].sum(axis=1)
    # 긴장비율 = Tension5~7 / 1~3
    d['tensionRate'] = d[['emotionTension5', 'emotionTension6', 'emotionTension7']].sum(axis=1) / d[['emotionTension1', 'emotionTension2', 'emotionTension3']].sum(axis=1)
    
    # 긍정감정의 임시 테이블
    pos_temp = d.filter(regex='Positive')
    # 가장 높은 긍정감정의 숫자(1~7)
    d['topPositive'] = pos_temp.idxmax(axis=1).apply(lambda x:int(x[-1]))
    # 가장 낮은 긍정감정의 숫자(1~7)
    d['botPositive'] = pos_temp.idxmin(axis=1).apply(lambda x:int(x[-1]))
    
    # 긍정 수치 1~7 * count 수
    for p, c in enumerate(pos_temp.columns):
        d.loc[:, [c]] = d.loc[:, [c]] * (p+1)
        
    del pos_temp
    
    # 긴장감정의 임시 테이블
    ten_temp = d.filter(regex='Tension')
    # 가장 높은 긴장감정의 숫자(1~7)
    d['topTension'] = ten_temp.idxmax(axis=1).apply(lambda x:int(x[-1]))
    # 가장 낮은 긴장감정의 숫자(1~7)
    d['botTension'] = ten_temp.idxmin(axis=1).apply(lambda x:int(x[-1]))
    
    # 긴장 수치 1~7 * count 수
    for t, c in enumerate(ten_temp.columns):
        d.loc[:, [c]] = d.loc[:, [c]] * (t+1)
    
    del ten_temp
    
    # 수치 반영 긍정 평균
    d['positiveWMean'] = d.filter(regex='Positive').mean(axis=1)
    # 수치 반영 긴장 평균
    d['tensionWMean'] = d.filter(regex='Tension').mean(axis=1)
    
    # 수치 반영 긍정 비율
    d['positiveWRate'] = d[['emotionPositive5', 'emotionPositive6', 'emotionPositive7']].sum(axis=1) / d[['emotionPositive1', 'emotionPositive2', 'emotionPositive3']].sum(axis=1)
    # 수치 반영 긴장 비율
    d['tensionWRate'] = d[['emotionTension5', 'emotionTension6', 'emotionTension7']].sum(axis=1) / d[['emotionTension1', 'emotionTension2', 'emotionTension3']].sum(axis=1)
    
    # 여기서부터는 그냥 막 만들어 본 피쳐들입니다...
    
    # 오전 컨디션에 긍정/긴장 반영 = 오전 컨디션 * (수치 반영 긍정 평균 + 수치 반영 긴장 평균) / 7 -> 수치가 1~7이니까 그냥 7로 나누어 봄ㅎㅎ..
    d['aCmET'] = round(d['amCondition'] * (d['positiveWMean'] + d['tensionWMean']) / 7)
    # 오전 감정에 긍정/긴장 반영 = 오전 감정 * (수치 반영 긍정 평균 + 수치 반영 긴장 평균) / 7 -> 이것도
    d['aEmET'] = round(d['amEmotion'] * (d['positiveWMean'] + d['tensionWMean']) / 7)
    # 오전 컨디션/감정에 긍정 긴장 반영한 것의 평균을 냄
    d['aCEmET'] = round((d['aCmET'] + d['aEmET']) / 2)
    
    # 오전 컨디션 긍정/긴장 반영에 오후 감정을 더한 뒤 평균을 냄
    d['aCmETpE'] = round((d['aCmET'] + d['pmEmotion']) / 2)
    # 오전 감정 긍정/긴장 반영에 오후 감정을 더한 뒤 평균을 냄
    d['aEmETpE'] = round((d['aEmET'] + d['pmEmotion']) / 2)
    # 오전 컨디션/감정 긍정/긴장 반영에 오후 감정을 더한 뒤 평균을 냄
    d['aCEmETpE'] = round((d['aCEmET'] + d['pmEmotion']) / 2)
    
    # 최고 수치를 반영한 오전 감정 = (오전 감정 * (최고 감정 + 최고 긴장)) / 14
    d['aCtPT'] = round(d['amEmotion'] * (d['topPositive'] + d['topTension']) / 14)
    # 최고 수치를 반영한 오전 컨디션 = (오전 감정 * (최고 감정 + 최고 긴장)) / 14
    d['aEtPT'] = round(d['amCondition'] * (d['topPositive'] + d['topTension']) / 14)
    # 최고 수치를 반영한 오후 감정 = (오전 감정 * (최고 감정 + 최고 긴장)) / 14
    d['pEtPT'] = round(d['pmEmotion'] * (d['topPositive'] + d['topTension']) / 14)
    
    # 둘 중에 하나로 쓰거나 컬럼명 변경해서 모두 써도 됩니다.
    # (최고 수치를 반영한 오전 컨디션 + 최고 수치를 반영한 오전 감정 + 최고 수치를 반영한 오후 감정) / 3
    # (최고 수치를 반영한 오전 컨디션 + 최고 수치를 반영한 오전 감정 + 최고 수치를 반영한 오후 감정)
#     d['aCEpEtPTm'] = round((d['aCtPT'] + d['aEtPT'] + d['pEtPT']) / 3)
    d['aCEpEtPTm'] = (d['aCtPT'] + d['aEtPT'] + d['pEtPT'])
    
    # 긍정적인지 = 수치 반영 긍정 평균이 중앙값보다 크면 1 아니면 0
    d['positive'] = d['positiveWRate'].apply(lambda x:1 if x > d['positiveWRate'].median() else 0)
    # 부정적인지 = 수치 반영 긍정 평균이 중앙값보다 작으면 1 아니면 0
    d['negative'] = d['positiveWRate'].apply(lambda x:1 if x < d['positiveWRate'].median() else 0)
    
    # 긴장 상태인지 = 수치 반영 긴장 평균이 중앙값보다 크면 1 아니면 0
    d['aroused'] = d['tensionWRate'].apply(lambda x:1 if x > d['tensionWRate'].median() else 0)
    # 편안한 상태인지 = 수치 반영 긴장 평균이 중앙값보다 작으면 1 아니면 0
    d['relaxed'] = d['tensionWRate'].apply(lambda x:1 if x < d['tensionWRate'].median() else 0)
    
    # 활동 비율 = 자전거, 도보의 합 / 운송수단, 가만히 있기의 합
    d['activityRate'] = d[['on_bicycle', 'on_foot']].sum(axis=1) / d[['in_vehicle', 'still']].sum(axis=1)
    # 0으로 나누기된 것을 0으로 치환
    d.replace([np.inf, -np.inf], 0, inplace=True)
    d.dropna(inplace=True)
    # random_state는 잘 나오는 것으로 바꿔보세요! 제가 해본 것 : 1111, 2222, 3333, 4444, 5555, 6666, 7777, 8888, 9999, 1234
    d.sample(n=d.shape[0], random_state=5555)
    
    return d

In [3]:
df1 = pd.read_csv(r'C:\lifelog_data_2018.csv')
df2 = pd.read_csv(r'C:\lifelog_data_2019.csv')
df3 = pd.read_csv(r'C:\lifelog_data_2020.csv')
for n, d in enumerate([df1, df2, df3]):
    globals() [f'df{n+1}'] = engineering(n, d.copy())

In [4]:
df = pd.concat([df1, df2, df3], axis=0)

In [5]:
# way 1
train = pd.DataFrame()
valid = pd.DataFrame()
test = pd.DataFrame()
# 스트레스 5종을 골고루 나누기 위함
for i in range(5):
    train = pd.concat([train, df.loc[df['pmStress'] == np.unique(df['pmStress'])[i]][:round(len(df) / 5 * 0.7)]], axis=0)
    valid = pd.concat([valid, df.loc[df['pmStress'] == np.unique(df['pmStress'])[i]][round(len(df) / 5 * 0.7):-round(len(df) / 5 * 0.1)]], axis=0)
    test = pd.concat([test, df.loc[df['pmStress'] == np.unique(df['pmStress'])[i]][-round(len(df) / 5 * 0.1):]], axis=0)

    # 동 수로 뽑을 때
#     temp = df.loc[df['pmStress'] == np.unique(df['pmStress'])[i]][:112]
#     train = pd.concat([train, temp[:round(112 * 0.7)]], axis=0)
#     valid = pd.concat([valid, temp[round(112 * 0.7):-round(112 * 0.1)]], axis=0)
#     test = pd.concat([test, temp[-round(112 * 0.1):]], axis=0)

train = train.sample(n=train.shape[0]) # , random_state=1234)
valid = valid.sample(n=valid.shape[0]) # , random_state=1234)
test = test.sample(n=test.shape[0]) # , random_state=1234)
print(train.shape, valid.shape, test.shape)

(667, 86) (299, 86) (105, 86)


In [6]:
df = pd.concat([train, test, valid], ignore_index=True, axis=0)

In [7]:
df = df[[c for c in df.columns if c not in ['userId', 'date']]]

In [8]:
# tabnet에서 사용되는 기법으로, index를 저장해두고 나중에 데이터셋을 나눔
train_indices = train.index
valid_indices = valid.index
test_indices = test.index

In [9]:
nunique = df.nunique()
types = df.dtypes

In [10]:
categorical_columns = []
categorical_dims =  {}
for col in df.columns:
    if types[col] == 'object' or nunique[col] <= 7:
        categorical_columns.append(col)
        categorical_dims[col] = len(np.unique(df[col]))

In [11]:
# target 제외한 피처 추가
features = [col for col in df.columns if col not in ['pmStress']] 
# 카테고리 피처 정보
cat_idxs = [i for i, f in enumerate(features) if f in categorical_columns]
cat_dims = [categorical_dims[f] for f in features if f in categorical_columns]

In [12]:
# 데이터셋 분할
X_train = df[features].values[train_indices]
y_train = df['pmStress'].values[train_indices].reshape(-1, 1)

X_valid = df[features].values[valid_indices]
y_valid = df['pmStress'].values[valid_indices].reshape(-1, 1)

X_test = df[features].values[test_indices]
y_test = df['pmStress'].values[test_indices].reshape(-1, 1)

In [13]:
print(categorical_columns, cat_idxs, cat_dims)

['gender', 'sleep_quality', 'dream', 'amCondition', 'amEmotion', 'pmEmotion', 'caffeine', 'alcohol', 'pmStress', 'topPositive', 'botPositive', 'topTension', 'botTension', 'aCtPT', 'aEtPT', 'pEtPT', 'positive', 'negative', 'aroused', 'relaxed'] [0, 2, 4, 5, 6, 7, 8, 9, 60, 61, 62, 63, 74, 75, 76, 78, 79, 80, 81] [2, 5, 4, 5, 5, 5, 2, 2, 7, 7, 7, 7, 6, 6, 6, 2, 2, 2, 2]


In [14]:
print(X_train.shape,
y_train.shape,
X_valid.shape,
y_valid.shape,
X_test.shape,
y_test.shape)

(667, 83) (667, 1) (299, 83) (299, 1) (105, 83) (105, 1)


In [18]:
# optimizer
of = optim.Adam
# optimizer params
op = dict(lr=0.01)
# steps
ns = 4
# n_d, n_a
nd = 24
# model gamma
g = 1.3
# step_size
ss = 10
# scheduler gamma
gm = 0.95
# mask_type
mt = 'entmax'
# batch size
bs = 16
# eval_metric
em = 'logloss'
# warm_start
ws = True

clf = TabNetMultiTaskClassifier(optimizer_fn=of, optimizer_params=op, scheduler_fn=optim.lr_scheduler.StepLR,
n_steps=ns, n_d=nd, n_a=nd, gamma=g,
scheduler_params={"step_size":ss, "gamma":gm}, 
mask_type=mt, verbose=0)

clf.fit(X_train.copy(), y_train.copy(),
        eval_set=[(X_valid.copy(), y_valid.copy())],
        loss_fn=[torch.nn.functional.cross_entropy]*5,
        max_epochs=30, 
        patience=0,
        batch_size=bs,
        eval_metric=[em],
        warm_start=ws)
preds = clf.predict(X_test.copy())
accuracy = accuracy_score(y_test.copy(), np.array(preds.copy()[0]).astype('float').reshape(-1, 1))
print("="*10, '\n', of, op, ns, nd, g, ss, gm, mt, bs, em, ws, '\n', 'accuracy :', accuracy)

 <class 'torch.optim.adam.Adam'> {'lr': 0.01} 4 24 1.3 10 0.95 entmax 16 logloss True 
 accuracy : 0.5428571428571428
